# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [11]:
#print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
data_file = os.path.join('..', 'WeatherPy', 'output_data', 'cities.csv')

read_city_df = pd.read_csv(data_file)
read_city_df

,City,ID,Country,Cloudiness,Temperature,Max Temperature,Latitude,Longitude,Humidity,Wind Speed
0,Souillac,933995,MU,75,66.58,66.58,-20.5167,57.5167,94,8.05
1,Vostok,2013279,RU,93,70.75,70.75,46.4856,135.8833,84,3.13
2,Saint George,5546220,US,1,82.60,87.53,37.1041,-113.5841,63,1.99
3,Dikson,1507390,RU,100,56.12,56.12,73.5069,80.5464,89,16.91
4,Ushuaia,3833367,AR,90,37.06,37.06,-54.8000,-68.3000,87,0.00
...,...,...,...,...,...,...,...,...,...,...
557,Kargasok,1504382,RU,100,57.43,57.43,59.0556,80.8572,66,6.49
558,Bulawayo,894701,ZW,0,54.01,54.01,-20.1500,28.5833,33,7.81
559,Nouadhibou,2377457,MR,1,76.98,76.98,20.9310,-17.0347,69,21.85
560,Tra Vinh,1563926,VN,26,79.05,79.05,9.9347,106.3453,87,7.85


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)

In [38]:
locations = read_city_df[["Latitude", "Longitude"]]
weight = read_city_df["Humidity"]

original_map = gmaps.figure()
add_heat_layer = gmaps.heatmap_layer(locations, weights=weight, max_intensity=100, point_radius=3, dissipating=False)

original_map.add_layer(add_heat_layer)
original_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
cleaned_df = read_city_df.loc[(read_city_df["Wind Speed"] <= 10) & (read_city_df["Cloudiness"] <= 0) & \
                                   (read_city_df["Max Temperature"] >= 70) & (read_city_df["Max Temperature"] <= 80)].dropna()
cleaned_df

,City,ID,Country,Cloudiness,Temperature,Max Temperature,Latitude,Longitude,Humidity,Wind Speed
27,Port Elizabeth,964420,ZA,0,77.29,77.29,-33.9180,25.5701,20,9.22
143,East London,1006984,ZA,0,74.34,74.34,-33.0153,27.9116,20,6.91
461,Xinyu,1788508,CN,0,79.93,79.93,27.8043,114.9334,83,6.53
525,Rovaniemi,638936,FI,0,76.41,76.41,66.5000,25.7167,46,9.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = cleaned_df.loc[:,["City","Country", "Latitude", "Longitude"]]
#hotel_df
# # Using DataFrame.insert() to add a column - from geeksforgeeks.org
# #df.insert(2, "Age", [21, 23, 24, 21], True)
hotel_df.insert(4, "Hotel Name", " ", True )
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
27,Port Elizabeth,ZA,-33.9180,25.5701,
143,East London,ZA,-33.0153,27.9116,
461,Xinyu,CN,27.8043,114.9334,
525,Rovaniemi,FI,66.5000,25.7167,


In [35]:
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }

        latitude = row['Latitude']
        longitude = row['Longitude']
        params['location'] = f"{latitude}, {longitude}"
        response_json = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
27,Port Elizabeth,ZA,-33.9180,25.5701,St Phillips Bed & Breakfast
143,East London,ZA,-33.0153,27.9116,Blue Lagoon Hotel
461,Xinyu,CN,27.8043,114.9334,Rongcheng Hotel
525,Rovaniemi,FI,66.5000,25.7167,Lapland Hotels Sky Ounasvaara


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
original_map.add_layer(markers)

# Display figure
original_map

Figure(layout=FigureLayout(height='420px'))